In [27]:
import pandas as pd
from pathlib import Path

In [28]:
data_dir = Path(".")
csv_dir = data_dir / "csv"
json_dir = data_dir / "json"

if not json_dir.exists():
    json_dir.mkdir()

In [29]:
csv_files = csv_dir.glob("*.csv")

In [30]:
stations = []

In [31]:
for file in csv_files:
    df = pd.read_csv(csv_dir / file.name)
    df.set_index(df.columns[0], inplace=True)

    # update stations
    for station in df.index:
        line = file.stem.rstrip("_inbound").rstrip("_outbound").rstrip("_Mon-Fri").rstrip("_Saturday")
        station_names = [station["name"].lower() for station in stations]
        if station.lower() not in station_names:
            stations.append({
                "name": station.lower(),
                "lines": [line.lower()],
            })
        else:
            for s in stations:
                if s["name"].lower() == station.lower():
                    if line.lower() not in s["lines"]:
                        s["lines"].append(line.lower())
        

    df.fillna("", inplace=True)
    df.to_json(json_dir / f"{file.stem}.json", orient='index')

In [32]:
with open(json_dir / "stations.json", "w") as f:
    f.write(str(stations))